In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [ ]:
# 1. Add the clean movie function that takes in the argument, "movie".

def clean_movie(movie):

    
    movie = dict(movie)
    
    alt_titles={}
    
    # now loop
    for key in ['Also known as','Arabice','Cantonese','Chinese','French','Hangul','Hebrew','Hepburn','Japanese'
               ,'Literally','Mandarin','McCune-Reischauer','Original title','Polish','Revised Romanization','Romanized'
                ,'Russian','Simplified','Traditional','Yiddish']:
        
        if key in movie:
            
            alt_titles[key]=movie[key]
            
            movie.pop(key)
    
    if len(alt_titles) > 0:
        movie['alt_titles']=alt_titles
 
    def change_column_name(old_name,new_name):
        if old_name in movie:
            movie[new_name]=movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    

    
    return(movie)

In [ ]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():
    
    kaggle_metadata=pd.read_csv('Resources/movies_metadata.csv',low_memory=False)
    ratings=pd.read_csv('Resources/ratings.csv')

    
    with open('Resources/wikipedia.movies.json', mode='r') as file:
        wiki_movies_raw=json.load(file)
    
    # 3. Write a list comprehension to filter out TV shows.
    wiki_tv_removal=[movie for movie in wiki_movies_raw if 'No. of episodes' not in movie]
    
    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    wiki_clean = [clean_movie(movie) for movie in wiki_tv_removal] 
                 
     # 5. Read in the cleaned movies list from Step 4 as a DataFrame. 
    wiki_clean_df = pd.DataFrame(wiki_clean)

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    

    
    try:
        
        
    except:

    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    # Drop any column that has over 90% NULLs
    wiki_columns_to_keep = [column for column in wiki_clean_df.columns if wiki_clean_df[column].isnull().sum() 
                          < len(wiki_clean_df)*0.9]
    
    # 8. Create a variable that will hold the non-null values from the “Box office” column. 
    box_office_7 = wiki_movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.    
    box_office_7=box_office_7.apply(lambda x: ' '.join(x) if type(x)==list else x)
    
    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\d+\.?\d*\s*[mbMB]illion'
   
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\d{1,3}(?:,\d{3})+'  
    
    # 12. Add the parse_dollars function.
    def parse_dollars(s):
        
        if type(s) != str:
            return np.nan

        if re.match(r'\$\s*\d+\.?\d*\s*milli?on',s,flags=re.IGNORECASE):
            s = re.sub('\$|\s|[a-zA-Z]','',s)
            value = float(s) * 10**6
            return value
    
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on',s,flags=re.IGNORECASE):
            s = re.sub('\$|\s|[a-zA-Z]','', s)
            value = float(s) * 10**9
            return value

        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)',s,flags=re.IGNORECASE):
            s = re.sub('\$|,','', s)
            value = float(s)
            return value

        else:
            return Nan
   
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office']=box_office_7.str.extract(f'({form_one}|{form_two})',flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.    
    budget_8 = wiki_movies_df['Budget'].dropna()
    budget_8 = budget_8.apply(lambda x: ' '.join(x) if type(x)==list else x)
    wiki_movies_df['budget']=budget_8.str.extract(f'({form_one}|{form_two})',flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Budget', axis=1, inplace=True)
    
     # 15. Clean the release date column in the wiki_movies_df DataFrame.   
    
    
    
    
    

In [ ]:


    

    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    return wiki_movies_df, kaggle_metadata, ratings 

In [ ]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = "Resources"
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia.movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [ ]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

In [ ]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [ ]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

In [ ]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()